In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data, datasets
import pandas as pd
from tqdm import tqdm

import spacy
import numpy as np

import random
import math
import time

from . import utils, helpers
import re
import os
import pickle

ImportError: attempted relative import with no known parent package

In [ ]:
DATASET_PATH = f".data/miguel"

In [ ]:
print("Reading and preprocessing Tabular dataset...")
data_fields = [('src', SRC), ('trg', TRG)]
train_data, val_data, test_data = data.TabularDataset.splits(path=f'{DATASET_PATH}/clean/',
                                                                     train='train.csv', validation='val.csv', test='test.csv',
                                                                     format='csv', fields=data_fields, skip_header=True)
   